# Welcome to Week 3!

## Agenda

### Part 1: Intro

1. A review of your own [Evals](https://docs.google.com/presentation/d/1OI-9yqJWK5hHNO06JDz44UduN4YmTf1HP-IRTtuAnig/edit?slide=id.p10#slide=id.p10)
2. What is an Agent?
3. Digital Twin Teams
4. First Agent labs - structured outputs (lab 2)

### Part 2: Agents

- Streaming (lab 3)
- Tools (lab 4)
- Interactions (lab 5)
- Coding (lab 6)
- Agent Loop (lab 7)

### Part 3: Team planning

1. Group and plan
2. Experiment!
3. Determine approach
4. Regroup


## New Teams for Digital Twin

The idea is that each of you builds the Digital Twin individually, but you collaborate to share your approach and leanings together, and to share tools.

**Team 1:**

Seb, Kinjal, Marylou, Dave

**Team 2:**

Jeff, Rohit, Salma

**Team 3:**

Sathiya, Ilya, Oscar, Josh


In [1]:
from agents import Agent, Runner
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [2]:
agent = Agent(name="Jokester")
response = await Runner.run(agent, "Tell me a joke about Agentic AI")
print(response.final_output)

Why did the agentic AI quit its day job?

Because it decided to *take matters into its own algorithms!*


# OpenAI Func Calling

In [13]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "sydney": "$2999"}

def get_ticket_prices(city: str) -> str:
    if city.lower() in ticket_prices:
        return ticket_prices[city.lower()]
    else:
        return "Sorry, we don't have ticket prices for that city."

get_ticket_prices(city="London")

'$799'

In [18]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_ticket_prices",
            "description": "Get the ticket price for flights to a specific city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The destination city name"
                    }
                },
                "required": ["city"]
            }
        }
    }
]

In [33]:
from dotenv import load_dotenv
from openai import OpenAI
import json
from IPython.display import Markdown, display

load_dotenv(override=True)
openai = OpenAI()

some_city = "mankato"
some_city = "london"

SYSTEM_PROMPT = f"you are an airline pricing expert.  keep your answers very brief.  if you don't know the answer, just say you don't know."
USER_PROMPT = f"You are looking to fly to {some_city}"

# messages = [{"role": "user", "content": message}]
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": USER_PROMPT}
    ]

response = openai.chat.completions.create(model="gpt-4o-mini",
                                          messages=messages,
                                          tools=tools,
                                          tool_choice="auto")
response_message = response.choices[0].message
messages.append(response_message)

# Check if the model wants to call a function
if response_message.tool_calls:
    for tool_call in response_message.tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        # Call your function
        if function_name == "get_ticket_prices":
            print(f"Calling function {function_name} with args {function_args}")
            function_response = get_ticket_prices(city=function_args.get("city"))
            
            # Add function response to messages
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": function_response
            })
        else:
            print(f"Function {function_name} not needed")
    
    # Second API call with function results
    final_response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    
    reply = final_response.choices[0].message.content
else:
    reply = response_message.content

display(Markdown(reply))


Calling function get_ticket_prices with args {'city': 'London'}


The ticket price to London is $799.